# Small experiment to train LSTM model and let it generate automatic text generation

In [3]:
from random import random
from numpy import array
from matplotlib import pyplot
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [4]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [5]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163815
Total Vocab:  60


In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  163715


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
163715/163715 [==============================] - 902s - loss: 2.5625   
Epoch 2/20
163715/163715 [==============================] - 1187s - loss: 2.5063  
Epoch 3/20
163715/163715 [==============================] - 1316s - loss: 2.4582  
Epoch 4/20
163715/163715 [==============================] - 1304s - loss: 2.4127  
Epoch 5/20
163715/163715 [==============================] - 1328s - loss: 2.3700  
Epoch 6/20
163715/163715 [==============================] - 1350s - loss: 2.3300  
Epoch 7/20
163715/163715 [==============================] - 1435s - loss: 2.2918  
Epoch 8/20
163715/163715 [==============================] - 1277s - loss: 2.2562  
Epoch 9/20
163715/163715 [==============================] - 1327s - loss: 2.2233  
Epoch 10/20
163715/163715 [==============================] - 1490s - loss: 2.1911  
Epoch 11/20
163715/163715 [==============================] - 1476s - loss: 2.1618  
Epoch 12/20
163715/163715 [==============================] - 1367s - loss: 2.1339  
E

#  Now that the LSTM model has been trained let's try to see how it generates text 

In [29]:
# start with random text
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Randomly choose a starting text pattern:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# LSTM model taking the random seed text now starting to generate the text
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\n Text generation done.")

Randomly choose a starting text pattern:
"  them hit her in the face. ‘i’ll put a stop to this,’
she said to herself, and shouted out, ‘you’d b "
eteer to tel the bormouse of the latter of the care tith the corrd, and she san oo anice, and she tent hnt lo the would bell aoong the was oo the tan  the wondd oo the was oow io whsh then a lottle saaten thee the was soink the was oo the tooe of the while 
a  ‘hh poen tie whst shael to tey an thee, and then soen io the would bell ao thle to an all dorn th the tond, bad aelen to tei toen i shilld toiee to soonk the was oo the tand oi the thnt of the harte haree herden the was oo the table, 
‘hele is as thl sieeg that she sas,’ the macc tirhle  she houkd aolng to the thitg sabbit, and she was notting an the could, and soon the harter sam the was oot in she woudd fo would hot theer hor hoad to the kaster, and the whnt hnrdle an the could, and soon th the woodd aod lottle thin she was oo the tand oi the thnt of the harte haree seid the whst of t

# The above model took almost 12 hours to be trained and the text generation as can be seen above is not quite that correct. This model can be trained more than 20 epochs 



In [30]:
#save the model and weights in Yaml format
from keras.models import model_from_json
from keras.models import model_from_yaml
import numpy
import os

numpy.random.seed(12345)

model.save('I:\\LSTM\\wonderland_model.h5')
model.save_weights('I:\\LSTM\\weights_model_yaml.h5')
# serialize model to YAML
model_yaml = model.to_yaml()
with open("I:\\LSTM\\wonderland_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("I:\\LSTM\\weights_model_yaml.h5")
print("Saved model to disk")

# load YAML and create model
yaml_file = open('I:\\LSTM\\wonderland_model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("I:\\LSTM\\weights_model_yaml.h5")
print("Loaded model from disk")
 


Saved model to disk
Loaded model from disk


In [31]:
#save the model and weights in json format
model_json = model.to_json()
with open("d:\\LSTM\\wonderland_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("d:\\LSTM\\wonderland_weigths_json.h5")
print("Saved model to disk")


json_file = open('d:\\LSTM\\wonderland_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("d:\\LSTM\\wonderland_weigths_json.h5")
print("Loaded model from disk")



Saved model to disk
Loaded model from disk
